# Use Trapezoid, Simpson's Rule, and Romberg integration

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Define a function to integrate

In [ ]:
def func(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return a*x**2 + b*x + c

# Define the integral to check answer

In [ ]:
def func_integral(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return (a*x**3)/3. + (b*x**2)/2. + c*x

# Define the core of the trapezoid method

In [ ]:
def trapezoid_core(f, x, h):
    return 0.5*h*(f(x+h) + f(x))

# Define wrapper function to perform trapezoid method

In [ ]:
def trapezoid_method(f,a,b,N):
    
    # f is function to integrate
    # a is lower limit
    # b is upper limit
    # N is the number of chunks to use
    
    # define x values for integral
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    # perform the integral using the trapezoid method
    for i in range(0,N-1,1):
        Fint += trapezoid_core(f,x[i],h)
    
    # return the answer
    return Fint

# Test the trapezoid method

In [ ]:
a = 0.0
b = 1.0

Answer = func_integral(b) - func_integral(a)
print("True answer = ",Answer)
error_prev = 1.0

# Trapezoidal approx
for N in range(10,100,10):
    trap_answer = trapezoid_method(func,a,b,N)
    error = (trap_answer - Answer)/Answer
    print("Trapezoid method ", trap_answer," Error ",error," improvement ",error_prev/error)
    error_prev = error

# Define the core of Simpson's method

In [ ]:
def simpsons_core(f, x, h):
    return h*( f(x) + 4.*f(x+h) + f(x+2*h))/3.

# Define a wrapper function for Simpson's method

In [ ]:
def simpsons_method(f,a,b,N):
    
    # f is function to integrate
    # a is lower limit
    # b is upper limit
    # N is the number of chunks to use
    
    # define x values for integral
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    # perform the integral using Simpson's method
    for i in range(0,N-2,2):
        Fint += simpsons_core(f,x[i],h)
        
    # apply simpson's over the last integral if N is even
    if ((N%2)==0):
        Fint += simpsons_core(f, x[-2],0.5*h)
    
    # return the answer
    return Fint

# Define the romberg integration core

In [ ]:
def romberg_core(f,a,b,i):
    
    h = b-a
    
    dh = h/2.**(i)
    
    K = h/2.**(i+1)
    
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
    
    return K*M

# Define a wrapper function for Romberg

In [ ]:
def romberg_integration(f,a,b,tol):
    
    i = 0
    
    imax = 1000
    
    # Define an error estimate, set to large value
    delta = 100.0*np.fabs(tol)
    
    # set an array of integral answers
    I = np.zeros(imax,dtype=float)
    
    # get zeroth romberg iteration
    I[0] = 0.5*(b-a)* (f(a) + f(b))
    
    # iterate by 1
    i += 1
    
    while(delta>tol):
        
        # find romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        # compute new fractional error estimate
        delta = np.fabs((I[i] - I[i-1])/I[i])
        
        print (i, I[i],I[i-1],delta)
        
        if (delta>tol):
            i+=1
            if(i>imax):
                print("max iterations reached")
                raise StopIteration('Stopping iterations after ',i)
    
    return I[i]

# check the integrals

In [ ]:
Answer = func_integral(1)-func_integral(0)

print("Answer = ",Answer)
print("Trapezoid")
print(trapezoid_method(func,0,1,10))
print("Simpson's method")
print(simpsons_method(func,0,1,10))
print("Romberg")
tolerance = 1.0e-4
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)